# Tracking

This page discusses the MLFlow component responsible for tracking - monitoring of the various attempts to train a model.

In [2]:
import mlflow
import numpy as np

import mlflow.models
import mlflow.sklearn
from mlflow.tracking import MlflowClient

from sklearn.linear_model import LinearRegression


client = MlflowClient()
mlflow.set_tracking_uri("file:///tmp/tracking_examples")

## Tags

Tags are key-value pairs used to systematize runs in MLflow. You can consider them as the properties of the run.

There are **system tags** that are defined by mlflow. These tags are listed in [System Tags Reference](https://mlflow.org/docs/latest/ml/tracking/tracking-api/#system-tags-reference) section of the official documentation.

Technically, you can set or change tags using:

- `mlflow.set_tag(<name>, <value>)`: for setting a single tag.
- `mlflow.set_tags(<dict>)`: for setting a set of tags with a single call.

---

The following cell shows a run and how tags can be applied to it.

In [ ]:
mlflow.set_experiment("tags_experiment")
with mlflow.start_run():
    mlflow.set_tag("single tag", "value for single tag")

    mlflow.set_tags({
        "tag1": "value1",
        "tag2": "value2"
    })

2025/08/10 16:12:37 INFO mlflow.tracking.fluent: Experiment with name 'tags_experiment' does not exist. Creating a new experiment.


🏃 View run likeable-sloth-308 at: http://localhost:5000/#/experiments/191345348810661561/runs/5c666b98b09349018ffa2d0c3061b080
🧪 View experiment at: http://localhost:5000/#/experiments/191345348810661561


The following cell searches for the previously created run. The specified `tag1`, `tag2` and `single_tag` are in the same line with, for example, `tags.mlflow.runName`, because at the storage level this information has the same status.

In [5]:
id = mlflow.get_experiment_by_name("tags_experiment").experiment_id
mlflow.search_runs(experiment_ids=id)

,run_id,experiment_id,status,artifact_uri,start_time,end_time,tags.mlflow.runName,tags.mlflow.source.type,tags.tag1,tags.mlflow.source.name,tags.tag2,tags.single tag,tags.mlflow.user
0,5c666b98b09349018ffa2d0c3061b080,191345348810661561,FINISHED,mlflow-artifacts:/191345348810661561/5c666b98b...,2025-08-10 13:12:37.702000+00:00,2025-08-10 13:12:37.719000+00:00,likeable-sloth-308,LOCAL,value1,/home/fedor/.virtualenvironments/python/lib/py...,value2,value for single tag,fedor


## Experiment

An experiment is a set of runs with different parameters for the same model. There are a few functions to manage the runs with very transparent names:

- `mlflow.create_experiment`.
- `mlflow.set_experiment`.

---

The following cell sets the "some experiment".

In [3]:
experiment = mlflow.set_experiment("some experiment")
experiment

<Experiment: artifact_location='mlflow-artifacts:/316075063118674621', creation_time=1754732470778, experiment_id='316075063118674621', last_update_time=1754732470778, lifecycle_stage='active', name='some experiment', tags={}>

Based on the log message, you can see that "some experiment" was created during the `set_experiment` call.

### Experiment object

There is an object that describes an experiment in mlflow. It has the type `mlflow.entities.experiment.Experiment`.

You can access the experiment objects in several cases:

- `mlflow.set_experiment`: returns the experiment that is active now.
- `mlflow.get_experiment`: returns the experiment with the specified id.
- `mlflow.get_experiment_by_name`: returns the experiment with the specified name.

---

The following cell shows the attributes of an experiment.

In [ ]:
experiment = mlflow.set_experiment("some experiment")
print(
    experiment.artifact_location,
    experiment.creation_time,
    experiment.experiment_id,
    experiment.last_update_time,
    experiment.lifecycle_stage,
    experiment.name,
    sep="\n"
)

mlflow-artifacts:/316075063118674621
1754732470778
316075063118674621
1754732470778
active
some experiment


### Current experiment

There is no special tool to get information about the experiment that is active in mlflow right now. The only option that is available now is to start a run and load information from it, using the `run.info.experiment_id` attributes to get the id of the experiment and the `mlflow.get_experiment` function to load the other information about the experiment.

---

The following cell shows how information about the currently used experiment can be loaded using the `experiment_id` that the run keeps.

In [2]:
with mlflow.start_run() as run:
    experiment_id = run.info.experiment_id
    print("Experiment ID:", experiment_id)
    experiment = mlflow.get_experiment(experiment_id)
    print("Experiment name:", experiment.name)

Experiment ID: 0
Experiment name: Default
🏃 View run secretive-horse-172 at: http://localhost:5000/#/experiments/0/runs/85fd8830eb3a435fbc84142f9083719c
🧪 View experiment at: http://localhost:5000/#/experiments/0


### Env variable

You can set the experiment to be used with `MLFLOW_EXPERIMENT_NAME` environment variable.

---

The following code sets the value `"env experiment"` for the `MLFLOW_EXPERIMENT_NAME` variable.

In [3]:
import os
import mlflow
os.environ["MLFLOW_EXPERIMENT_NAME"] = "env experiment"

The next code, by loading information about the active run, shows that the value from the environment variable was applied.

In [4]:
with mlflow.start_run() as run:
    name = mlflow.get_experiment(run.info.experiment_id).name
print(name)

env experiment


## Run

Run in mlflow refers to the execution of a machine learning experiment or a piece of code that you want to track and record. It represents a specific execution instance with associated metadata and recorded metrics, parameters, artifacts, and tags.

The most usefull methods of `mflow` in roder to manipulate the runs are listted in the following table:

| Method | Description |
|--------|-------------|
| `mlflow.start_run` | To start run |
| `mlflow.end_run` | To end run |
| `mlflow.delete_run` | To delete run |
| `mlflow.search_runs` | To search for the runs |

Note that one runs can be nested to the orhter - child nad parent runs.

For more details check out:

- [Run management](https://mlflow.org/docs/latest/ml/tracking/tracking-api/#run-management) section of the official mlflow tutorials - it describes functions to manipulate runs.
- [Runs](tracking/runs.ipynb) page in this website.

---

Run can be started with `mlflow.start_run()` and end it with `mlflow.end_run()`. By passing the `run_name` parameter to the `mlflow.start_run()` function, you can define the name of the run. The following cell demonstrates how it may be:

In [10]:
import mlflow

mlflow.start_run(run_name="my_run_name")
mlflow.end_run()

🏃 View run my_run_name at: http://localhost:5000/#/experiments/0/runs/925288ded96849d38de4ab46b5f9e3e4
🧪 View experiment at: http://localhost:5000/#/experiments/0


**Note** it's OK to have several runs with the same name - you can think of them as different versions of the same run.

The most popular approach to start a run is using `mlflow.start_run` as a context manager expression. The run begins upon entering the context and ends upon exiting it.

The follwoing cell shows launching an empty run using exactly context manager approach.

In [18]:
with mlflow.start_run(run_name="with run"):
    pass

🏃 View run with run at: http://localhost:5000/#/experiments/101046082978810346/runs/b96f254b78f14e7ca8e19b2ec23146f9
🧪 View experiment at: http://localhost:5000/#/experiments/101046082978810346


Use the `mlflow.search_runs` command to list all available runs.

In [17]:
mlflow.search_runs(search_all_experiments=True)

,run_id,experiment_id,status,artifact_uri,start_time,end_time,tags.mlflow.source.name,tags.mlflow.source.type,tags.mlflow.runName,tags.mlflow.user
0,7ac6c477dc724ea5ad07065cdae6794e,101046082978810346,FINISHED,mlflow-artifacts:/101046082978810346/7ac6c477d...,2025-08-08 09:12:47.469000+00:00,2025-08-08 09:12:47.480000+00:00,/home/fedor/.virtualenvironments/python/lib/py...,LOCAL,with run,fedor
1,925288ded96849d38de4ab46b5f9e3e4,0,FINISHED,mlflow-artifacts:/0/925288ded96849d38de4ab46b5...,2025-08-08 08:46:34.027000+00:00,2025-08-08 08:46:34.041000+00:00,/home/fedor/.virtualenvironments/python/lib/py...,LOCAL,my_run_name,fedor
2,c243c89b84fc4c62ad756d4512d6f4ee,0,FINISHED,mlflow-artifacts:/0/c243c89b84fc4c62ad756d4512...,2025-08-08 08:41:22.880000+00:00,2025-08-08 08:44:31.008000+00:00,/home/fedor/.virtualenvironments/python/lib/py...,LOCAL,my_run_name,fedor


## Log

The following table represents things you can log with mlflow and their corresponding functions.

| **Loggable Entity**         | **MLflow Function(s)**                                                                                                                                |
| --------------------------- | ----------------------------------------------------------------------------------------------------------------------------------------------------- |
| Parameters (single)         | `mlflow.log_param(key, value)`                                                                                                                        |
| Parameters (multiple)       | `mlflow.log_params(dict_of_params)`                                                                                                                   |
| Metrics (single)            | `mlflow.log_metric(key, value, step=None)`                                                                                                            |
| Metrics (multiple)          | Metric logging is usually done one at a time — there's no built-in `log_metrics()`, but you can loop over a dict to call `log_metric` multiple times. |
| Artifacts (single file)     | `mlflow.log_artifact(local_path, artifact_path=None)`                                                                                                 |
| Artifacts (directory)       | `mlflow.log_artifacts(local_dir, artifact_path=None)`                                                                                                 |
| Models (generic)            | `mlflow.log_model(model, artifact_path, **kwargs)`                                                                                                    |
| Models (framework-specific) | - `mlflow.sklearn.log_model(...)`<br>- `mlflow.pytorch.log_model(...)`<br>- `mlflow.tensorflow.log_model(...)`, etc.                                  |

---

The following code represents the procedure of model logging.

In [8]:
model = LinearRegression().fit(
    X=np.random.normal(0, 2, size=(100, 2)),
    y=np.random.normal(0, 20, size=100)
)
signature = mlflow.models.infer_signature(
    np.random.normal(0, 2, size=(100, 2)),
    np.random.normal(0, 20, size=100)
)

with mlflow.start_run(run_name="test_run"):
    mlflow.sklearn.log_model(model, signature=signature)

🏃 View run test_run at: http://localhost:5000/#/experiments/155825779609388391/runs/57a4997e6e7d4b37b4237f1254ffd2f7
🧪 View experiment at: http://localhost:5000/#/experiments/155825779609388391
